# What is in this notebook

1. Sets up the environment (PyTorch, Hugging Face, bitsandbytes, PEFT).

2. Loads a pretrained LLaVA-Med model.

3. Runs inference on a sample fundus image + question.

4. Provides a LoRA fine-tuning scaffold so you can start small experiments with your own dataset.

# How to use this notebook

1. Run each cell step by step.

2. Replace the sample fundus image with your dataset (e.g., from Kaggle).

3. Extend the LoRA fine-tuning block with a proper Trainer (Hugging Face has ready templates).

# Next upgrade steps

* Connect your fundus dataset (EyePACS/APTOS/Messidor) in Colab.

* Write a dataset loader that pairs (image, question, answer).

* Use Hugging Face Trainer for supervised fine-tuning with LoRA.

* Add Gradio demo in Colab for interactive Q&A.

In [ ]:
# =======================================
# STEP 0: Environment Setup
# =======================================

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers datasets accelerate bitsandbytes peft pillow gradio

Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of gradio to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.13.0
    Uninstalling gradio_client-1.13.0:
      Successfully uninstalled gradio_client-1.13.0
  Attempting uninstall: gradio
    Found existing installation: g

In [ ]:
!pip install --upgrade transformers

In [ ]:
# Install bleeding-edge versions (support llava_mistral arch)
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes pillow

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-939kdln0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-939kdln0
  Resolved https://github.com/huggingface/transformers.git to commit 97ca0b47124c82cfde886450ea56160ad45c4153
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ymikouel
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ymikouel
  Resolved https://github.com/huggingface/peft.git to commit 6030f9160ed2fc17220f6f41382a66f1257b6a93
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-req-build-0dg2fam8
  Runni

In [ ]:
# =======================================
# STEP 1: Import Libraries
# =======================================
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor, pipeline
from peft import LoraConfig, get_peft_model
from PIL import Image
import requests

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoModelForVision2Seq

model_id = "microsoft/llava-med-v1.5-mistral-7b"  # Example, replace if needed

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForVision2Seq.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True
)

processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/auto/modeling_auto.py:2279: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `llava_mistral` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
# =======================================
# STEP 2: Load Pretrained LLaVA-Med
# (using Hugging Face checkpoint)
# =======================================
# Example checkpoint (you can replace with latest)
# Some LLaVA-Med weights are hosted by Microsoft or community repos
model_id = "microsoft/llava-med-v1.5-mistral-7b"  # Example, replace if needed

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=True  # Quantized loading (saves VRAM)
)
processor = AutoProcessor.from_pretrained(model_id)

ValueError: The checkpoint you are trying to load has model type `llava_mistral` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [ ]:
# =======================================
# STEP 3: Run Inference (Fundus Image + Question)
# =======================================

# Sample fundus image (replace with your dataset)
url = "https://upload.wikimedia.org/wikipedia/commons/3/35/Fundus_photograph_of_normal_left_eye.jpg"
image = Image.open(requests.get(url, stream=True).raw).convert("RGB")

question = "Does this fundus image show signs of diabetic retinopathy?"

inputs = processor(images=image, text=question, return_tensors="pt").to(device, torch.float16)

# Generate answer
output = model.generate(**inputs, max_new_tokens=64)
answer = tokenizer.decode(output[0], skip_special_tokens=True)

print("Question:", question)
print("Answer:", answer)

In [ ]:
# =======================================
# STEP 4: LoRA Fine-tuning Scaffold
# (You can extend this with your own dataset)
# =======================================

# Configure LoRA for lightweight fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],  # typical for LLaVA-style adapters
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA adapters
model = get_peft_model(model, lora_config)

# Dummy dataset (replace with real fundus+QA pairs)
from datasets import Dataset
data = {
    "image": [url],
    "question": ["What is the diabetic retinopathy grade?"],
    "answer": ["No DR"]
}
dataset = Dataset.from_dict(data)

print("Dataset ready:", dataset)

# Training loop placeholder (to be extended)
# Use Hugging Face Trainer or Accelerate for full fine-tuning